In [4]:
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI, OpenAIError

In [5]:
client = OpenAI()

## Interact with api

In [6]:
from openai import OpenAI, OpenAIError

client = OpenAI()

try:
    response = client.responses.create(
        model="gpt-4o-mini",
        input="Tell a joke about the moon"
    )
    print(response.output_text)
except OpenAIError as e:
    print(f"OpenAI API error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")


Why did the moon break up with the sun?

Because it needed some space! 🌙✨


In [8]:
messages = [
    {"role": "user", "content": "tell me a joke about Alexey"}
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=messages
)


In [10]:
print(response.output_text)

Why did Alexey bring a ladder to the bar?

Because he heard the drinks were on the house!


In [9]:
print(response.model_dump_json(indent=2))

{
  "id": "resp_053fd613543851bd0068e722a7a2088193a4f3912b078484be",
  "created_at": 1759978151.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4o-mini-2024-07-18",
  "object": "response",
  "output": [
    {
      "id": "msg_053fd613543851bd0068e722a8898c8193bebc2fbd2149325d",
      "content": [
        {
          "annotations": [],
          "text": "Why did Alexey bring a ladder to the bar?\n\nBecause he heard the drinks were on the house!",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [],
  "top_p": 1.0,
  "background": false,
  "conversation": null,
  "max_output_tokens": null,
  "max_tool_calls": null,
  "previous_response_id": null,
  "prompt": null,
  "prompt_cache_key": null,
  "reasoning": {
    "effort": nu

What is interesting here:

output - everything that the LLM returned

usage - how many tokens we used (also - how much we paid)

Getting the text:

response.output[0].content[0].text

response.output_text

## Stream response

In [11]:
stream = client.responses.create(
    model='gpt-4o-mini',
    input=messages,
    stream=True
)

for event in stream:
    print(event)


ResponseCreatedEvent(response=Response(id='resp_05fa3cc314f49d6d0068e723437320819492c912adfed90937', created_at=1759978307.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='auto', status='in_progress', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='disabled', usage=None, user=None, store=True), sequence_number=0, type='response.created')
ResponseInProgressEvent(response=Response(id='resp_05fa3cc314f49d6d0068e723437320819492c912adfed90937', created_at=1759978307.0, error=None, incomplete_details=None, instru

In [13]:
stream = client.responses.create(
    model='gpt-4o-mini',
    input=messages,
    stream=True
)
for event in stream:
    if hasattr(event, 'delta'):
        print(event.delta, end='')

Why did Alexey bring a ladder to the bar?

Because he heard the drinks were on the house!

## System and User Prompt

In [14]:
system_prompt = """
You're an assistant that can make jokes. Always find out the name of
the person to make the jokes personalized. Once you know the name,
make the joke about them.
""".strip()

messages = [
    {"role": "developer", "content": system_prompt},
    {"role": "user", "content": "tell me a joke"}
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=messages
)

print(response.output_text)


Sure! But first, what's your name?


In [15]:
messages.append(response.output[0])
messages.append({"role": "user", "content": "Nelson"})

response = client.responses.create(
    model='gpt-4o-mini',
    input=messages
)

print(response.output_text)

Great to meet you, Nelson! Here’s a joke just for you:

Why did Nelson bring a ladder to the bar?

Because he heard the drinks were on the house!


##  Save and continue chat history using ToyAi

It will keep track of the previous conversation and send it back to the API:

In [16]:
from toyaikit.llm import OpenAIClient
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIResponsesRunner

In [17]:
system_prompt = """
you're an assistant that can make jokes. Always find out the name of
the person to make the jokes personalized. Once you know the name,
make the joke about them.
""".strip()

llm_client = OpenAIClient(
    model='gpt-4o-mini',
    client=client
)

runner = OpenAIResponsesRunner(
    tools=None,
    developer_prompt=system_prompt,
    chat_interface=IPythonChatInterface(),
    llm_client=llm_client
)

runner.run();

KeyboardInterrupt: Interrupted by user

## Conversational Chat using langchain

In [26]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [27]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

# Keeps the full history in RAM for this Python process
memory = ConversationBufferMemory(return_messages=True)

chain = ConversationChain(llm=llm, memory=memory, verbose=False)


C:\Users\nelso\AppData\Local\Temp\ipykernel_30204\2409486733.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_30204\2409486733.py:6: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(llm=llm, memory=memory, verbose=False)


In [28]:

print(chain.predict(input="Hi, I'm Nelson. I love NBA fantasy."))
print(chain.predict(input="What did I say my name is?"))
print(chain.predict(input="And what hobby did I mention?"))


Hi Nelson! It's great to meet you! NBA fantasy is such an exciting way to engage with the league. Do you have a favorite team or player that you like to build your fantasy roster around? Or maybe you have some strategies you swear by for drafting and managing your team?
You mentioned that your name is Nelson! It's nice to meet you, Nelson.
You mentioned that you love NBA fantasy! It's a fun way to get involved with the league and track your favorite players. Do you have any specific players or strategies you like to focus on in your fantasy leagues?


## Conversational Chat using LECL

In [ ]:
from typing import Dict
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

In [34]:

# 1️⃣ Define the LLM and prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful and concise assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
chain = prompt | llm

# 2️⃣ In-memory history store keyed by session_id
store: Dict[str, BaseChatMessageHistory] = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 3️⃣ Wrap the chain with memory
chat_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 4️⃣ Run a conversation
cfg = {"configurable": {"session_id": "user_123"}}

In [35]:
resp1 = chat_with_memory.invoke({"input": "Hi, I'm Nelson from Houston who loves NBA fantasy."}, cfg)
print(resp1.content)

Hi Nelson! It's great to meet a fellow NBA fantasy enthusiast. How can I assist you with your fantasy basketball needs?


In [36]:
resp2 = chat_with_memory.invoke({"input": "What hobby did I say I like?"}, cfg)
print(resp2.content)

You mentioned that you love NBA fantasy.


In [37]:
# Optional: see stored messages
history = get_session_history("user_123")
for m in history.messages:
    who = "User" if isinstance(m, HumanMessage) else "Assistant"
    print(f"{who}: {m.content}")

User: Hi, I'm Nelson from Houston who loves NBA fantasy.
Assistant: Hi Nelson! It's great to meet a fellow NBA fantasy enthusiast. How can I assist you with your fantasy basketball needs?
User: What hobby did I say I like?
Assistant: You mentioned that you love NBA fantasy.
